**Dataset**

In [1]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.vgg16 import preprocess_input

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
train_data_path='/kaggle/input/newemotiondataset/Image_frame_crop_face_vgg16/train'
val_data_path='/kaggle/input/newemotiondataset/Image_frame_crop_face_vgg16/val'
test_data_path= '/kaggle/input/newemotiondataset/Image_frame_crop_face_vgg16/test_fix/test'
real_test_path='/kaggle/input/real-test-emotion/test'

train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   horizontal_flip=True,
                                   vertical_flip=True,
                                   fill_mode='nearest',
                                   rescale=1./255
)

validation_datagen = ImageDataGenerator(preprocessing_function=preprocess_input,
                                        rescale=1./255
)

test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input,
                                  rescale=1./255
)
real_test_datagen=ImageDataGenerator(rescale=1./255)

In [3]:
train_set=train_datagen.flow_from_directory(train_data_path,
                                            target_size=(224, 224),
                                            batch_size = 32,
                                            subset = 'training',
                                            class_mode= 'categorical')

Found 25997 images belonging to 7 classes.


In [4]:
validate_set=validation_datagen.flow_from_directory(val_data_path,
                                            target_size=(224, 224),
                                            batch_size = 32,
                                                    shuffle=True,
                                            class_mode= 'categorical')

Found 6763 images belonging to 7 classes.


In [5]:
test_set=test_datagen.flow_from_directory(test_data_path,
                                            target_size=(224, 224),
                                            batch_size = 32,
                                            class_mode= 'categorical')

Found 6745 images belonging to 7 classes.


In [6]:
real_test_set=real_test_datagen.flow_from_directory(real_test_path,
                                                   target_size=(224,224),
                                                    batch_size=32,
                                                    class_mode='categorical'
                                                   )

Found 7178 images belonging to 7 classes.


**Train**

In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sn
import skimage.io
import keras.backend as K
import tensorflow as tf
from datetime import datetime
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dense, Flatten, Dropout,BatchNormalization ,Activation
from tensorflow.keras.models import Model, Sequential
from keras.applications.nasnet import NASNetLarge
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, EarlyStopping
from tensorflow.keras.optimizers import Adam

In [7]:
image_size=[224,224]

In [8]:
base_model = tf.keras.applications.VGG16(input_shape=image_size + [3],include_top=False,weights="imagenet")
base_model.output

58889256/58889256 [==============================] - 2s 0us/step


<KerasTensor: shape=(None, 7, 7, 512) dtype=float32 (created by layer 'block5_pool')>

In [9]:
for layer in base_model.layers[:-10]:
    layer.trainable=False

In [10]:
for (i,layer) in enumerate(base_model.layers):
  print(str(i) + " " + base_model.layers[i].__class__.__name__ + " " + str(base_model.layers[i].trainable))

0 InputLayer False
1 Conv2D False
2 Conv2D False
3 MaxPooling2D False
4 Conv2D False
5 Conv2D False
6 MaxPooling2D False
7 Conv2D False
8 Conv2D False
9 Conv2D True
10 MaxPooling2D True
11 Conv2D True
12 Conv2D True
13 Conv2D True
14 MaxPooling2D True
15 Conv2D True
16 Conv2D True
17 Conv2D True
18 MaxPooling2D True


In [11]:
base_model.layers

In [12]:
model=Sequential()
model.add(base_model)
model.add(Flatten())
model.add(BatchNormalization())
model.add(Dense(64,kernel_initializer='he_uniform'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(64,kernel_initializer='he_uniform'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(64,kernel_initializer='he_uniform'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dense(7,activation='softmax'))

In [13]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 7, 7, 512)         14714688  
                                                                 
 flatten (Flatten)           (None, 25088)             0         
                                                                 
 batch_normalization (BatchN  (None, 25088)            100352    
 ormalization)                                                   
                                                                 
 dense (Dense)               (None, 64)                1605696   
                                                                 
 batch_normalization_1 (Batc  (None, 64)               256       
 hNormalization)                                                 
                                                                 
 activation (Activation)     (None, 64)                0

In [14]:
def f1_score(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    return f1_val

In [15]:
METRICS = [
      tf.keras.metrics.BinaryAccuracy(name='accuracy'),
      tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Recall(name='recall'),
      tf.keras.metrics.AUC(name='auc'),
        f1_score,
]

In [16]:
lrd = ReduceLROnPlateau(monitor = 'val_loss',patience = 20,verbose = 1,factor = 0.50, min_lr = 1e-10)

mcp = ModelCheckpoint('model_vgg16_100diem.h5')

es = EarlyStopping(monitor = 'val_loss',verbose=1, patience=20)

In [19]:
model.compile(optimizer=Adam(learning_rate=0.00001),
              loss='categorical_crossentropy',
              metrics=METRICS)

In [ ]:
history=model.fit(train_set,
                  validation_data=validate_set,
                  epochs = 80,
                  verbose = 1,
                  callbacks=[lrd,mcp,es])

Epoch 1/5
179/813 [=====>........................] - ETA: 5:15 - loss: 1.9318 - accuracy: 0.8503 - precision: 0.2328 - recall: 0.0208 - auc: 0.6240 - f1_score: 0.0236

In [ ]:
test_accuracy = model.evaluate(test_set)

In [ ]:
real_test_accuracy=model.evaluate(real_test_set)

In [ ]:
def Train_Val_Plot(acc,val_acc,loss,val_loss,auc,val_auc,precision,val_precision,recall,val_recall,f1,val_f1):

    fig, (ax1, ax2,ax3,ax4,ax5,ax6) = plt.subplots(1,6, figsize= (20,6))
    fig.suptitle(" MODEL'S METRICS VISUALIZATION ")

    ax1.plot(range(1, len(acc) + 1), acc)
    ax1.plot(range(1, len(val_acc) + 1), val_acc)
    ax1.set_title('History of Accuracy')
    ax1.set_xlabel('Epochs')
    ax1.set_ylabel('Accuracy')
    ax1.legend(['training', 'validation'])


    ax2.plot(range(1, len(loss) + 1), loss)
    ax2.plot(range(1, len(val_loss) + 1), val_loss)
    ax2.set_title('History of Loss')
    ax2.set_xlabel('Epochs')
    ax2.set_ylabel('Loss')
    ax2.legend(['training', 'validation'])

    ax3.plot(range(1, len(auc) + 1), auc)
    ax3.plot(range(1, len(val_auc) + 1), val_auc)
    ax3.set_title('History of AUC')
    ax3.set_xlabel('Epochs')
    ax3.set_ylabel('AUC')
    ax3.legend(['training', 'validation'])

    ax4.plot(range(1, len(precision) + 1), precision)
    ax4.plot(range(1, len(val_precision) + 1), val_precision)
    ax4.set_title('History of Precision')
    ax4.set_xlabel('Epochs')
    ax4.set_ylabel('Precision')
    ax4.legend(['training', 'validation'])
    
    ax5.plot(range(1, len(recall) + 1), recall)
    ax5.plot(range(1, len(val_recall) + 1), val_recall)
    ax5.set_title('History of Recall')
    ax5.set_xlabel('Epochs')
    ax5.set_ylabel('Recall')
    ax5.legend(['training', 'validation'])

    ax6.plot(range(1, len(f1) + 1), f1)
    ax6.plot(range(1, len(val_f1) + 1), val_f1)
    ax6.set_title('History of F1-score')
    ax6.set_xlabel('Epochs')
    ax6.set_ylabel('F1 score')
    ax6.legend(['training', 'validation'])


    plt.show()
Train_Val_Plot(history.history['accuracy'],history.history['val_accuracy'],
               history.history['loss'],history.history['val_loss'],
               history.history['auc'],history.history['val_auc'],
               history.history['precision'],history.history['val_precision'],
               history.history['recall'],history.history['val_recall'],
               history.history['f1_score'],history.history['val_f1_score']
              )